In [63]:
import sqlite3
from sqlite3 import Error
import pandas as pd

In [68]:
def create_connection(db_file):
    """ create a database connection to the SQLite database
        specified by db_file
    :param db_file: database file
    :return: Connection object or None
    """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        return conn
    except Error as e:
        print(e)

    return conn

In [69]:
def create_table(conn, create_table_sql):
    """ create a table from the create_table_sql statement
    :param conn: Connection object
    :param create_table_sql: a CREATE TABLE statement
    :return:
    """
    try:
        c = conn.cursor()
        c.execute(create_table_sql)
    except Error as e:
        print(e)

In [98]:
def main():
    database = "../db/pythonsqlite.db"
    
    sql_create_meetings_table = """ CREATE TABLE IF NOT EXISTS meetings (
                                        id text PRIMARY KEY,
                                        meeting_type text NOT NULL,
                                        location text,
                                        datetime_iso text,
                                        cancelled boolean,
                                        agenda_url text,
                                        video_url text,
                                        minutes_filename text
                                    ); """

    sql_create_votes_table = """CREATE TABLE IF NOT EXISTS votes (
                                    id integer PRIMARY KEY,
                                    meeting_id integer NOT NULL,
                                    title text,
                                    carried boolean,
                                    for text,
                                    against text,
                                    FOREIGN KEY (meeting_id) REFERENCES meetings (id)
                                );"""

    # create a database connection
    conn = create_connection(database)

    # create tables
    if conn is not None:
        create_table(conn, sql_create_meetings_table)
        create_table(conn, sql_create_votes_table)
    else:
        print("Error! cannot create the database connection.")

In [99]:
main()

In [100]:
cnx = create_connection("../db/pythonsqlite.db")



In [101]:
meetings = pd.read_json("../generated_data/all_meeting_data.json")

In [102]:
meetings.head()

,id,meeting_type,location,datetime_iso,agenda_url,cancelled,video_url,minutes_filename
0,e8697880-f06f-4be5-abfd-ea0a4f69dcf7,Accessibility Advisory Committee Meeting,Electronic Meeting,2022-10-03T16:00:00,https://pub-peterborough.escribemeetings.com/M...,False,,None
1,f3dc4a44-970a-4246-96c8-7b4731b59c33,Accessibility Advisory Committee Meeting,Electronic Meeting,2022-09-07T16:00:00,https://pub-peterborough.escribemeetings.com/M...,False,,2022-09-07.Accessibility-Advisory-Committee-Me...
2,660c01bb-fe4d-49a0-855e-6d67b2ee1fa3,Accessibility Advisory Committee Meeting,Electronic Meeting,2022-06-01T16:00:00,https://pub-peterborough.escribemeetings.com/M...,False,,2022-06-01.Accessibility-Advisory-Committee-Me...
3,89365d4a-1a96-43a6-b8fe-44c2f6b0b3e0,Accessibility Advisory Committee Meeting,Electronic Meeting,2022-05-04T16:00:00,https://pub-peterborough.escribemeetings.com/M...,False,,2022-05-04.Accessibility-Advisory-Committee-Me...
4,3d098fd5-753f-4079-a014-b0428e837b80,Accessibility Advisory Committee Meeting,Electronic Meeting,2022-04-06T16:00:00,https://pub-peterborough.escribemeetings.com/M...,False,,2022-04-06.Accessibility-Advisory-Committee-Me...


In [103]:
query=f''' insert or replace into meetings ({",".join(meetings.columns)}) values ({",".join(['?' for _ in meetings.columns])}) '''
cnx.executemany(query, meetings.to_records(index=False))
cnx.commit()

In [90]:
meetings["cancelled"][0]

False

In [92]:
dtypes = {"id": str, "meeting_type": str, "location": str, "datetime_iso": str, "agenda_url": str, "cancelled": bool, "video_url": str, "minutes_filename": str}

In [94]:
meetings = meetings.astype(dtypes)

In [104]:
df = pd.read_sql_query("SELECT * FROM meetings", cnx)


In [105]:
df

,id,meeting_type,location,datetime_iso,cancelled,agenda_url,video_url,minutes_filename
0,e8697880-f06f-4be5-abfd-ea0a4f69dcf7,Accessibility Advisory Committee Meeting,Electronic Meeting,2022-10-03T16:00:00,b'\x00',https://pub-peterborough.escribemeetings.com/M...,,None
1,f3dc4a44-970a-4246-96c8-7b4731b59c33,Accessibility Advisory Committee Meeting,Electronic Meeting,2022-09-07T16:00:00,b'\x00',https://pub-peterborough.escribemeetings.com/M...,,2022-09-07.Accessibility-Advisory-Committee-Me...
2,660c01bb-fe4d-49a0-855e-6d67b2ee1fa3,Accessibility Advisory Committee Meeting,Electronic Meeting,2022-06-01T16:00:00,b'\x00',https://pub-peterborough.escribemeetings.com/M...,,2022-06-01.Accessibility-Advisory-Committee-Me...
3,89365d4a-1a96-43a6-b8fe-44c2f6b0b3e0,Accessibility Advisory Committee Meeting,Electronic Meeting,2022-05-04T16:00:00,b'\x00',https://pub-peterborough.escribemeetings.com/M...,,2022-05-04.Accessibility-Advisory-Committee-Me...
4,3d098fd5-753f-4079-a014-b0428e837b80,Accessibility Advisory Committee Meeting,Electronic Meeting,2022-04-06T16:00:00,b'\x00',https://pub-peterborough.escribemeetings.com/M...,,2022-04-06.Accessibility-Advisory-Committee-Me...
...,...,...,...,...,...,...,...,...
1609,555aef2b-0dc2-4db2-8cd7-196b53f055f6,Waste Management Committee Meeting,"Doris Room, City Hall",2010-09-20T14:00:00,b'\x00',https://pub-peterborough.escribemeetings.com/M...,,None
1610,390c4fb7-1495-46f0-880f-22c04f8a762f,Waste Management Committee Meeting,"Doris Room, City Hall",2010-03-08T14:00:00,b'\x00',https://pub-peterborough.escribemeetings.com/M...,,2010-03-08.Waste-Management-Committee-Meeting.pdf
1611,f602ac27-e786-44ef-9fbc-3d939dc99323,Waste Management Committee Meeting,"Doris Room, City Hall",2009-12-07T14:00:00,b'\x00',https://pub-peterborough.escribemeetings.com/M...,,2009-12-07.Waste-Management-Committee-Meeting.pdf
1612,9c803554-42fd-48c1-b32c-2b7f8e02b77b,Waste Management Committee Meeting,"Doris Room, City Hall",2009-11-16T14:00:00,b'\x00',https://pub-peterborough.escribemeetings.com/M...,,2009-11-16.Waste-Management-Committee-Meeting.pdf


In [ ]:
d